In [1]:
import pandas as pd

In [4]:
df = pd.read_excel('datasets/FoodInsecurity_Hispanic_Demographics_Tone_Preferences_Dataset.xlsx')

In [5]:
df.head(2)

,age,gender,ethnicity,race,education,marital_status,income,employment,language,disability,states,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8
0,45-54,female,non hispanic,native american,High School,na,"$25,000 - $49,999",Employed Part time,both,i do not have a disability,indiana,Persuasive,Simplier,Empathetic,Persuasive,Original,Original,Persuasive,Original
1,18-24,male,hispanic,white,High School,single,"Less than $25,000",Employed Part time,english,i do not have a disability,illinois,Original,Simplier,Empathetic,Simplier,Simplier,Original,Original,Persuasive


In [6]:
df.columns

Index(['age', 'gender', 'ethnicity', 'race', 'education', 'marital_status',
       'income', 'employment', 'language', 'disability', 'states', 'sample_1',
       'sample_2', 'sample_3', 'sample_4', 'sample_5', 'sample_6', 'sample_7',
       'sample_8'],
      dtype='object')

In [16]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Assuming df is your DataFrame
categorical_features = ['age', 'gender', 'ethnicity', 'race', 'education', 'marital_status', 'income', 'employment', 'language', 'disability', 'states']

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' avoids multicollinearity

# Fit and transform the categorical features
encoded_categorical = encoder.fit_transform(df[categorical_features])

# Convert to DataFrame for easier handling
encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_features))

# Combine with the rest of the data
df_encoded = pd.concat([df.drop(columns=categorical_features), encoded_df], axis=1)

In [17]:
df_encoded.head(2)

,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,age_25-34,age_35-44,...,language_na,language_spanish,disability_na,states_illinois,states_indiana,states_kansas,states_na,states_new York,states_new jersey,states_texas
0,Persuasive,Simplier,Empathetic,Persuasive,Original,Original,Persuasive,Original,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Original,Simplier,Empathetic,Simplier,Simplier,Original,Original,Persuasive,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Encode tone preferences
tone_preferences = ['sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5', 'sample_6', 'sample_7', 'sample_8']
df_encoded_tones = pd.get_dummies(df[tone_preferences])

# Combine with the rest of the data
df_encoded = pd.concat([df_encoded, df_encoded_tones], axis=1)


In [24]:
df_encoded.drop(['sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5', 'sample_6', 'sample_7', 'sample_8'], axis=1, inplace=True)

In [25]:
df_encoded.head(2)

,age_25-34,age_35-44,age_45-54,age_55-64,age_65,age_na,gender_male,gender_non binary,ethnicity_non hispanic,race_Mexican,...,sample_6_Persuasive,sample_6_Simplier,sample_7_Empathetic,sample_7_Original,sample_7_Persuasive,sample_7_Simplier,sample_8_Empathetic,sample_8_Original,sample_8_Persuasive,sample_8_Simplier
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,False,False,False,False,True,False,False,True,False,False
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,True,False


In [27]:
# Compute correlation matrix
correlation_matrix = df_encoded.corr()
correlation_matrix

,age_25-34,age_35-44,age_45-54,age_55-64,age_65,age_na,gender_male,gender_non binary,ethnicity_non hispanic,race_Mexican,...,sample_6_Persuasive,sample_6_Simplier,sample_7_Empathetic,sample_7_Original,sample_7_Persuasive,sample_7_Simplier,sample_8_Empathetic,sample_8_Original,sample_8_Persuasive,sample_8_Simplier
age_25-34,1.000000,-0.206284,-0.433629,-0.240772,-0.166667,-0.166667,-0.166667,0.174964,0.083333,0.089087,...,0.204124,0.059131,-0.218681,-0.063758,0.354787,-0.039421,-0.018615,-0.170020,0.019118,0.140028
age_35-44,-0.206284,1.000000,-0.134176,-0.074501,-0.051571,-0.051571,0.137523,-0.036092,0.137523,-0.110264,...,-0.126323,0.475716,-0.024290,0.100834,-0.134176,0.069121,0.042239,0.100834,-0.141975,0.007221
age_45-54,-0.433629,-0.134176,1.000000,-0.156608,-0.108407,-0.108407,0.157683,-0.075869,-0.236525,0.031607,...,-0.024140,-0.282051,0.181032,-0.123156,-0.048951,-0.048951,-0.094660,0.128182,-0.185397,0.153181
age_55-64,-0.240772,-0.074501,-0.156608,1.000000,-0.060193,-0.060193,0.060193,-0.042126,0.210675,0.072393,...,-0.147442,0.021356,0.072903,-0.138159,-0.156608,0.199320,-0.174792,0.245617,0.179520,-0.202289
age_65,-0.166667,-0.051571,-0.108407,-0.060193,1.000000,-0.041667,-0.166667,-0.029161,0.250000,-0.089087,...,0.153093,-0.108407,0.050465,0.170020,-0.108407,-0.108407,0.111687,-0.095637,-0.114708,0.078766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sample_7_Simplier,-0.039421,0.069121,-0.048951,0.199320,-0.108407,0.137973,-0.236525,-0.075869,-0.137973,0.163303,...,-0.144841,0.300699,-0.415777,-0.248825,-0.282051,1.000000,0.235550,0.002513,-0.185397,-0.053820
sample_8_Empathetic,-0.018615,0.042239,-0.094660,-0.174792,0.111687,0.111687,-0.111687,-0.084678,-0.111687,0.114424,...,-0.182384,0.235550,0.005636,-0.159034,-0.094660,0.235550,1.000000,-0.277716,-0.333096,-0.406622
sample_8_Original,-0.170020,0.100834,0.128182,0.245617,-0.095637,-0.095637,0.042505,-0.066932,-0.063758,0.221519,...,-0.234261,-0.123156,0.062205,-0.084011,0.002513,0.002513,-0.277716,1.000000,-0.263286,-0.321403
sample_8_Persuasive,0.019118,-0.141975,-0.185397,0.179520,-0.114708,-0.114708,0.305888,-0.080279,0.210298,-0.245256,...,0.070244,-0.072350,-0.054028,0.102389,0.153744,-0.185397,-0.333096,-0.263286,1.000000,-0.385496
